# Développement d'un Agent IA pour la Résolution d'un Puzzle 3x3 (8 Puzzle)

Dans ce projet, nous allons créer un agent IA pour résoudre un puzzle de 3 lignes et 3 colonnes, où chaque case contient un chiffre de 1 à 8, sans répétition, avec une case vide. L'objectif de l'agent est de déplacer les cases en utilisant la case vide afin d'obtenir l'ordre croissant de 1 à 8.

## Environnemment

| 1 | 2 | 3 |
|---|---|---|
| 4 | 5 | 6 |
|   7| 8 |  |


### Type de l'environnement 
1) Entièrement observable: l'agent a accès à toutes les informations nécessaires pour connaître l'état du puzzle dès la première observation.
2) Déterministe : chaque action de l'agent entraine un état attendu, sans variations inattendues.
3) Statique: l'état du l'environnement ne se change pas de manière autonome. c'est l'action executé par l'agent qui modifie la position des cases.
4) Discret: dans le puzzle 3x3, il y a un nombre limité de cases et de configurations possibles pour les chiffres et la case vide. Chaque mouvement mène à un état distinct et identifiable du puzzle.
5) Connu : l’agent sait que chaque action déplace la case vide dans une direction précise et produit un état bien défini, sans aucune incertitude.
6) Séquentiel: les actions sont donc séquentielles car elles s'enchaînent logiquement et influencent les états futurs de l’environnement.
7) Agent unique : il n’y a qu'un seul agent qui manipule les cases pour atteindre l’objectif. Aucun autre agent ne participe à la résolution du puzzle.  

### Representation de l'evironnement :
L'état du puzzle 3X3 peut être représenter par une matrice 2D, chaque cellule contient un nombre de 1 à 8 et une case vide représenter par 0. 

In [ ]:
# Exemple d'état initial
initial_state = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 0, 8]  # 0 représente la case vide
]

### Test de solvabilité

Il n'est pas possible de résoudre une instance de 8 puzzle si le nombre d'inversions est impair dans l'état d'entrée.

| 1 | 2 | 3 |
|---|---|---|
| 4 | 5 | 6 |
|   | 8 | 7 |


Écrivez-le de façon linéaire, 1,2,3,4,5,6,8,7 - Ne tenez pas compte de la tuile vide.

Trouvez maintenant le nombre d'inversions, en comptant les tuiles qui précèdent une autre tuile avec un nombre inférieur.

Dans notre cas, 1,2,3,4,5,6,7 a 0 inversion, et 8 a 1 inversion car il précède le chiffre 7.

Le nombre total d'inversions est de 1 (nombre impair) et le puzzle est donc insoluble.


-----------------------------------------------------------------------------------------------------------------------------------------------



| 5 | 2 | 8 |
|---|---|---|
| 4 | 1 | 7 |
|   | 3 | 6 |

5 précède 1,2,3,4 - 4 inversions

2 précède 1 - 1 inversion

8 précède 1,3,4,6,7 - 5 inversions

4 précèdent 1,3 - 2 inversions

1 précède aucun - 0 inversions

7 précède 3,4 - 2 inversions

3 précède aucun - 0 inversions

6 précède aucun - 0 inversions

total des inversions 4+1+5+2+0+2+0+0 = 14 (nombre pair)
Cette énigme peut donc être résolue.

## Class Environnemt

La classe Environnement représente l'état d'un puzzle 3x3 et contient deux méthodes principales :

<span style="color: orange; font-weight: 700">is_goal</span> vérifie si l'état actuel correspond à l'état objectif (cases ordonnées de 1 à 8 avec une case vide).

<span style="color: orange; font-weight: 700">is_solvable</span> détermine si le puzzle est résolvable en comptant les inversions, ce qui indique la possibilité de résoudre l'état initial.

In [2]:
class Environnement:

    def __init__(self, initial_state):
        # Initialisation de l'environnement par un état de départ.
        self.state = initial_state

    # Vérifier si l'état actuel est l'état objective
    def is_goal(self, state):
        goal_state = [
            [1, 2, 3],
            [4, 5, 6],
            [7, 8, 0]
        ]
        return state == goal_state

    def is_solvable(self):
        # Aplatir l'état et retirer le zéro (case vide)
        flat_puzzle = [num for row in self.state for num in row if num != 0]
        inversions = 0

        # Compter les inversions
        for i in range(len(flat_puzzle)):
            for j in range(i + 1, len(flat_puzzle)):
                if flat_puzzle[i] > flat_puzzle[j]:
                    inversions += 1

        # Le puzzle est solvable si :
        # - le nombre d'inversions est pair
        return inversions % 2 == 0

## Test class Environnement

In [ ]:
initial_state = [
    [1, 2, 3],
    [4, 5, 6],
    [0, 8, 7]
]
environnement = Environnement(initial_state)
assert environnement.is_solvable() == False, f"Erreur l'état doit être insoulvable"
print(f"l'état founrie n'est pas soulvable")

## Class Capteur

La classe Capteur fournit à l'agent les informations sur l'état actuel du puzzle et la position de la case vide en accédant directement à l'environnement. Elle permet ainsi à l'agent de prendre des décisions en fonction de la configuration courante des cases.

In [11]:
class Capteur:

    def __init__(self, environnement):
        self.environnement = environnement

    # Lire l'état actuel
    def get_current_state(self):
        return self.environnement.state

    # Trouver la position de la case vide
    def find_empty(self, state):
        for i, row in enumerate(state):
            if 0 in row:
                # print(f"la position de vide et {i, row.index(0)}")
                return i, row.index(0)

### Test de la class Capteur

In [12]:
# Étape 1 : Créer un état initial du puzzle
initial_state = [
    [1, 2, 3],
    [4, 0, 5],  # 0 représente la case vide position (1,1)
    [6, 7, 8]
]

# Créer l'Environnement
environnement = Environnement(initial_state)

# Créer le capteur
capteur = Capteur(environnement)

# Étape 4 : Tester la méthode lire_etat()
etat = capteur.get_current_state()
assert etat == initial_state, f"Erreur : l'état lu est {etat}, attendu {initial_state}"

# Étape 5 : Tester la méthode position_vide()
position_vide = capteur.find_empty(etat)
assert position_vide == (
    1, 1), f"Erreur : position vide est {position_vide}, attendu (1, 1)"

print(f"Etat initial est : {initial_state}")

print(f"position vide est : {position_vide}")

print("Tous les tests du capteur ont réussi !")

Etat initial est : [[1, 2, 3], [4, 0, 5], [6, 7, 8]]
position vide est : (1, 1)
Tous les tests du capteur ont réussi !
